In [57]:
import pandas as pd
import numpy as np

arrow = u'\N{BLACK RIGHT-POINTING TRIANGLE}'


#Read stratigraphic chart and make strat chart easily searchable
chart = pd.read_excel('ChronostratChart2018-08.xlsx')
chart = chart.applymap(lambda s:s.strip().lower() if type(s) == str else s)

#Have a look?
#chart

In [52]:
def get_time(age, 
        chart=chart, 
        gauge = 0.5,
        early=0.75, 
        mid=0.5, 
        late=0.25, 
        latest = 0, 
        earliest = 1,
        late_labels = None,
        early_labels = None,
        mid_labels = None, 
        use_descriptive = True,
        m = True, 
        age_col = 7, 
        uncertainty = False, 
        uncertainty_col = 9):
    '''Function to convert name of startigraphic age to years in million years
    age -- string with geological age
    chart -- The International Chronostratigraphic Chart as pandas data array
    gauge -- a scalafr to set time from 0 (min for period) to 1 (max for period)
    mid, late, latest, early, earliest = predefined gauges
    use_descriptive -- True if prefixes defined in labels should change returned time
    m -- rerturne Ma, else years
    age_col -- colomn on chart that contains age in Ma
    uncertainty -- if uncertainty of period definitions should be included
        False if not, 'min' for min age and 'max' for max age
    '''
    
    in_phrase = age.strip().lower().split()
    chart = np.array(chart)
    find = np.zeros((chart.shape[0], len(in_phrase))).astype('bool')
    
    if late_labels==None:
        late_labels = ['upper', 'later', 'late']
    
    if early_labels==None:
        early_labels = ['lower', 'early', 'earlier']
        
    if mid_labels==None:
        mid_labels = ['mid', 'middle']
        
    for i, in_p in enumerate(in_phrase):
        find[:,i] = np.any(chart == in_p, axis=1)
        
    here = np.sum(find, axis=1)
    max_here = np.max(here)
    ii = np.where(here==max_here)[0]
    
    if max_here == 1 and len(in_phrase) == 2 and use_descriptive:
        print('Non standard expression.')
        if in_phrase[0] in late_labels:
            gauge = late
        elif in_phrase[0] in ['latest']:
            gauge = latest
        elif in_phrase[0] in early_labels:
            gauge = early
        elif in_phrase[0] in ['earliest']:
            gauge = earliest
        elif in_phrase[0] in mid_labels:
            gauge = mid
        else:
            print('Unknown prefix.')
    
    if uncertainty==False:
        e_top = e_bottom = 0
    elif uncertainty=='max':
        e_top = np.nan_to_num(- chart[ii-1, uncertainty_col][0])
        e_bottom = np.nan_to_num(chart[ii, uncertainty_col][-1])
    elif uncertainty=='min':
        e_top = np.nan_to_num(chart[ii-1, uncertainty_col][0])
        e_bottom = np.nan_to_num(- chart[ii, uncertainty_col][-1])
        
          
    min_my = chart[ii-1, age_col][0]-e_top # ii-1 because start of next time is end of this
    max_my = chart[ii, age_col][-1]+e_bottom
    my = min_my + gauge*(max_my-min_my)
    
    # Option to return years instead of Ma
    if not m:
        my *= 1e6
        
    return my, min_my, max_my
    

In [58]:
tests = ['sandbian', 'phanerozoic', 'meghalayan', 'upper cretaceous', 'lower ordovician'] 
    
for test in tests: 
    print(test, arrow, get_time(test) )

sandbian ▶ (455.7, 453.0, 458.4)
phanerozoic ▶ (270.5, 0.0, 541.0)
meghalayan ▶ (0.0021, 0.0, 0.0042)
upper cretaceous ▶ (83.25, 66.0, 100.5)
lower ordovician ▶ (477.7, 470.0, 485.4)


In [64]:
for test in tests: 
    print(test, arrow, get_time(test, uncertainty='max'), get_time(test, uncertainty='min')  )

sandbian ▶ (456.5, 453.7, 459.29999999999995) (454.9, 452.3, 457.5)
phanerozoic ▶ (271.0, 0.0, 542.0) (270.0, 0.0, 540.0)
meghalayan ▶ (0.0021, 0.0, 0.0042) (0.0021, 0.0, 0.0042)
upper cretaceous ▶ (83.25, 66.0, 100.5) (83.25, 66.0, 100.5)
lower ordovician ▶ (479.34999999999997, 471.4, 487.29999999999995) (476.05, 468.6, 483.5)


In [63]:
#upper case
print(get_time('PERMIAN'))
#mix
print(get_time('ARcheAN'))

(275.40099999999995, 251.902, 298.9)
(3250.0, 2500.0, 4000.0)


In [74]:
print(get_time('latest Cretaceous'))
print(get_time('late Cretaceous'))
print(get_time('early Cretaceous'))
print(get_time('earliest Cretaceous'))
print(get_time('Cretaceous'))
print(get_time('mid Cretaceous'))

Non standard expression.
(66.0, 66.0, 145.0)
Non standard expression.
(85.75, 66.0, 145.0)
Non standard expression.
(125.25, 66.0, 145.0)
Non standard expression.
(145.0, 66.0, 145.0)
(105.5, 66.0, 145.0)
Non standard expression.
(105.5, 66.0, 145.0)
